In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from bs4 import BeautifulSoup
import requests
import re
from datetime import datetime
# CREATING FUNCTION to get each fight urls from a fighter's URL
def get_fighter_fight_urls(fighter_url):
    page = requests.get(fighter_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    fight_urls = []
    
    fight_rows = soup.find_all('tr', class_='b-fight-details__table-row')
    for fight_row in fight_rows:
        onclick_value = fight_row.get('onclick')
        if onclick_value:
            match = re.search(r"'(http://www.ufcstats.com/fight-details/.+?)'", onclick_value)
            if match:
                fight_link = match.group(1)
                fight_urls.append(fight_link)
    
    return fight_urls


#Scrape fighter statistics
import datetime
def get_numeric_value(value_str):
    try:
        return float(value_str)
    except ValueError:
        return None

def convert_height(height_str):
    height_str = height_str.replace('"', '')
    feet, inches = map(int, height_str.split("'"))
    total_inches = feet * 12 + inches
    return total_inches

def convert_weight(weight_str):
    return float(weight_str.replace('lbs.', '').strip())

def convert_reach(reach_str):
    if reach_str == '--':
        return None
    return float(reach_str.replace('"', '').strip())

def convert_percentage(percentage_str):
    try:
        if isinstance(percentage_str, str):
            numeric_value = float(percentage_str.replace('%', '')) / 100
            return numeric_value
        elif isinstance(percentage_str, float):
            return percentage_str  # Already a float, no conversion needed
    except ValueError:
        return None

def fighter_stats(fighter_url):
    fighter_page = requests.get(fighter_url)
    fighter_soup = BeautifulSoup(fighter_page.content, 'html.parser')

    fighter_data = {}
    fighter_physical_stats = fighter_soup.find('div', class_='b-list__info-box')
    if fighter_physical_stats:
        physical_details = fighter_physical_stats.find_all('li', class_='b-list__box-list-item')
        for detail in physical_details:
            label = detail.find('i', class_='b-list__box-item-title').get_text(strip=True)
            value = detail.get_text(strip=True).replace(label, '').strip()

            if label == 'Height:':
                fighter_data[label] = convert_height(value)
            elif label == 'Weight:':
                fighter_data[label] = convert_weight(value)
            elif label == 'Reach:':
                fighter_data[label] = convert_reach(value)
            else:
                fighter_data[label] = value

    fighter_career_stats = fighter_soup.find('div', class_='b-list__info-box-left')
    if fighter_career_stats:
        career_stats = fighter_career_stats.find_all('li', class_='b-list__box-list-item')
        for stat in career_stats:
            label_element = stat.find('i', class_='b-list__box-item-title')
            value = stat.get_text(strip=True).replace(label_element.get_text(strip=True), '').strip()
            label = label_element.get_text(strip=True).rstrip(':')

            if label in ['Str. Acc.', 'Str. Def', 'TD Acc.', 'TD Def.']:
                new_label = f"{label} (%)"
                converted_value = convert_percentage(value)
                fighter_data[new_label] = converted_value
            else:
                fighter_data[label] = value

    cols_to_strip_percent = ['Str. Acc. (%)', 'Str. Def (%)', 'TD Acc. (%)', 'TD Def. (%)']
    fighter_df = pd.DataFrame([fighter_data])
    fighter_df[cols_to_strip_percent] = fighter_df[cols_to_strip_percent].replace('%', '', regex=True)
    
    
    fighter_df['DOB:'] = pd.to_datetime(fighter_df['DOB:'], format='%b %d, %Y')
    today = datetime.datetime.today()
    fighter_df['Age'] = (today - fighter_df['DOB:']).apply(lambda x: x.days // 365)
    fighter_df.drop(columns=['DOB:'], inplace=True)

    fighter_df.drop('', axis=1, inplace=True)
    return fighter_df


def get_opponent_stats(fighter_url):
    opponent_urls = []

    fighter_page = requests.get(fighter_url)
    fighter_soup = BeautifulSoup(fighter_page.content, 'html.parser')

    recent_fights_table = fighter_soup.find('table')
    recent_fights_rows = recent_fights_table.find_all('tr')

    for row in recent_fights_rows:
        fighter_column = row.find('td', class_='b-fight-details__table-col l-page_align_left')
        
        if fighter_column:
            fighter_links = fighter_column.find_all('a', class_='b-link b-link_style_black')
            if len(fighter_links) >= 2:
                opponent_url = fighter_links[1]['href']
                opponent_urls.append(opponent_url)
    
    opponent_stats_dfs = []
    for opponent_url in opponent_urls:
        opponent_stats_df = fighter_stats(opponent_url)
        opponent_stats_dfs.append(opponent_stats_df)
    
    combined_opponent_stats = pd.concat(opponent_stats_dfs, ignore_index=True)
    combined_opponent_stats.columns = ["Opponent " + col for col in combined_opponent_stats.columns]
    return combined_opponent_stats


def scrape_fight_data(fight_url):   
    fight_page = requests.get(fight_url)
    fight_soup = BeautifulSoup(fight_page.content, 'html.parser')

    data = []
    fight_table = fight_soup.find('tbody', class_='b-fight-details__table-body')

    fighter_divs = fight_soup.find_all('div', class_='b-fight-details__person')
    outcomes = [outcome_tag.get_text(strip=True) if (outcome_tag := fighter_div.find('i', class_='b-fight-details__person-status')) else None for fighter_div in fighter_divs]

    for row in fight_table.find_all('tr', class_='b-fight-details__table-row'):
        fighter_names = row.find_all('a', class_='b-link_style_black')
        fighters = [name.get_text(strip=True) for name in fighter_names]

        kd = [col.get_text(strip=True) for col in row.find_all('td')[1].find_all('p')]
        sig_str = [col.get_text(strip=True) for col in row.find_all('td')[2].find_all('p')]
        sig_str_percent = [col.get_text(strip=True) for col in row.find_all('td')[3].find_all('p')]
        total_str = [col.get_text(strip=True) for col in row.find_all('td')[4].find_all('p')]
        td = [col.get_text(strip=True) for col in row.find_all('td')[5].find_all('p')]
        td_percent = [col.get_text(strip=True) for col in row.find_all('td')[6].find_all('p')]
        sub_att = [col.get_text(strip=True) for col in row.find_all('td')[7].find_all('p')]
        rev = [col.get_text(strip=True) for col in row.find_all('td')[8].find_all('p')]
        ctrl = [col.get_text(strip=True) for col in row.find_all('td')[9].find_all('p')]

        for i in range(len(fighters)):
            fight_info = {
                'Fighter': fighters[i],
                'KD': kd[i],
                'Sig. str.': sig_str[i],
                'Sig. str. %': sig_str_percent[i],
                'Total str.': total_str[i],
                'Td': td[i],
                'Td %': td_percent[i],
                'Sub. att': sub_att[i],
                'Rev.': rev[i],
                'Ctrl': ctrl[i],
                'Fight URL': fight_url,
                'Outcome': outcomes[i] 
            }
            data.append(fight_info)

    df = pd.DataFrame(data)

    new_data = []
    new_columns = [
        'Fighter', 'Opponent', 'KD', 'Opponent KD', 'Sig. str.', 'Opponent Sig. str.',
        'Sig. str. %', 'Opponent Sig. str. %', 'Total str.', 'Opponent Total str.',
        'Td', 'Opponent Td', 'Td %', 'Opponent Td %', 'Sub. att', 'Opponent Sub. att',
        'Rev.', 'Opponent Rev.', 'Ctrl', 'Opponent Ctrl', 'Outcome'
    ]

    for i in range(0, len(df), 2):
        fighter_row = df.iloc[i]
        opponent_row = df.iloc[i + 1]

        new_row = [
            fighter_row['Fighter'], opponent_row['Fighter'],
            fighter_row['KD'], opponent_row['KD'],
            fighter_row['Sig. str.'], opponent_row['Sig. str.'],
            fighter_row['Sig. str. %'], opponent_row['Sig. str. %'],
            fighter_row['Total str.'], opponent_row['Total str.'],
            fighter_row['Td'], opponent_row['Td'],
            fighter_row['Td %'], opponent_row['Td %'],
            fighter_row['Sub. att'], opponent_row['Sub. att'],
            fighter_row['Rev.'], opponent_row['Rev.'],
            fighter_row['Ctrl'], opponent_row['Ctrl'],
            fighter_row['Outcome']
        ]

        new_data.append(new_row)

    fight_df = pd.DataFrame(new_data, columns=new_columns)

    fight_df.replace('---', None, inplace=True)


    try:
        fight_df['Sig. str. %'] = fight_df['Sig. str. %'].apply(convert_percentage)
        fight_df['Opponent Sig. str. %'] = fight_df['Opponent Sig. str. %'].apply(convert_percentage)
        fight_df['Td %'] = fight_df['Td %'].apply(convert_percentage)
        fight_df['Opponent Td %'] = fight_df['Opponent Td %'].apply(convert_percentage)
    except AttributeError:
        pass  


    fight_df[['Sig. Landed', 'Sig. Attempted']] = fight_df['Sig. str.'].str.split(' of ', expand=True)
    fight_df['Sig. Landed'] = fight_df['Sig. Landed'].astype(int)
    fight_df['Sig. Attempted'] = fight_df['Sig. Attempted'].astype(int)

    fight_df[["Opponent Sig. Landed", "Opponent Sig. Attempted"]] = fight_df["Opponent Sig. str."].str.split(' of ', expand=True)
    fight_df["Opponent Sig. Landed"] = fight_df["Opponent Sig. Landed"].astype(int)
    fight_df["Opponent Sig. Attempted"] = fight_df["Opponent Sig. Attempted"].astype(int)

    fight_df[['Total Landed', 'Total Attempted']] = fight_df['Total str.'].str.split(' of ', expand=True)
    fight_df['Total Landed'] = fight_df['Total Landed'].astype(int)
    fight_df['Total Attempted'] = fight_df['Total Attempted'].astype(int)

    fight_df[['Opponent Total Landed', 'Opponent Total Attempted']] = fight_df['Opponent Total str.'].str.split(' of ', expand=True)
    fight_df['Opponent Total Landed'] = fight_df['Opponent Total Landed'].astype(int)
    fight_df['Opponent Total Attempted'] = fight_df['Opponent Total Attempted'].astype(int)

    fight_df[['Td Landed', 'Td Attempted']] = fight_df['Td'].str.split(' of ', expand=True)
    fight_df['Td Landed'] = fight_df['Td Landed'].astype(int)
    fight_df['Td Attempted'] = fight_df['Td Attempted'].astype(int)

    fight_df[['Opponent Td Landed', 'Opponent Td Attempted']] = fight_df['Opponent Td'].str.split(' of ', expand=True)
    fight_df['Opponent Td Landed'] = fight_df['Opponent Td Landed'].astype(int)
    fight_df['Opponent Td Attempted'] = fight_df['Opponent Td Attempted'].astype(int)

    fight_df.drop(['Sig. str.', "Opponent Sig. str.", 'Total str.', 'Opponent Total str.', 'Td', 'Opponent Td'], axis=1, inplace=True)

    return fight_df


#CREATE FUNCTION TO SCRAPE PAST FIGHT RESULST FROM A FIGHTER'S URL PAGE
def past_fights(fighter_url):
    fighter_page = requests.get(fighter_url)
    fighter_soup = BeautifulSoup(fighter_page.content, 'html.parser')

    # Get searched fighter's name
    fighter_name = fighter_soup.find('span', class_='b-content__title-highlight').get_text(strip=True)
    
    #Create DF from all previous fights from searched fighter
    fight_urls = get_fighter_fight_urls(fighter_url)
    all_fight_dfs = []
    for fight_url in fight_urls:
        fight_df = scrape_fight_data(fight_url)
        all_fight_dfs.append(fight_df)

    combined_df = pd.concat(all_fight_dfs, ignore_index=True)

    #Add the fight information to DF
    fight_table = fighter_soup.find('tbody', class_='b-fight-details__table-body')

    each_fight_details = fight_table.find_all('tr')

    for idx, fight in enumerate(each_fight_details[1:]):
        event_info_elements = fight.find_all('td', class_='b-fight-details__table-col l-page_align_left')
        round_time_elements = fight.find_all('td', class_='b-fight-details__table-col')

        event_name = event_info_elements[1].find_all('p')[0].text.strip()
        event_date = event_info_elements[1].find_all('p')[1].text.strip()

        try:
            method_of_victory = event_info_elements[2].find('p').text.strip()
        except AttributeError:
            method_of_victory = None
        
        rounds_element = round_time_elements[-2].find('p')
        if rounds_element:
            rounds = rounds_element.text.strip()
        else:
            rounds = None

        time_element = round_time_elements[-1].find('p')
        if time_element:
            time = time_element.text.strip()
        else:
            time = None

        combined_df.at[idx, 'Event Name'] = event_name
        combined_df.at[idx, 'Event Date'] = event_date
        combined_df.at[idx, 'Method of Victory'] = method_of_victory
        combined_df.at[idx, 'Rounds'] = rounds
        combined_df.at[idx, 'Time'] = time

    # Flip the data from Opponent to Fighter if the searched fighter is Opponent
    for index, row in combined_df.iterrows():
        if row['Opponent'] == fighter_name:
            combined_df.at[index, 'Fighter'] = row['Opponent']
            combined_df.at[index, 'Opponent'] = row['Fighter']

            combined_df.at[index, 'KD'] = row['Opponent KD']
            combined_df.at[index, 'Opponent KD'] = row['KD']

            combined_df.at[index, 'Sig. str. %'] = row['Opponent Sig. str. %']
            combined_df.at[index, 'Opponent Sig. str. %'] = row['Sig. str. %']

            combined_df.at[index, 'Td %'] = row['Opponent Td %']
            combined_df.at[index, 'Opponent Td %'] = row['Td %']

            combined_df.at[index, 'Sub. att'] = row['Opponent Sub. att']
            combined_df.at[index, 'Opponent Sub. att'] = row['Sub. att']

            combined_df.at[index, 'Rev.'] = row['Opponent Rev.']
            combined_df.at[index, 'Opponent Rev.'] = row['Rev.']

            combined_df.at[index, 'Ctrl'] = row['Opponent Ctrl']
            combined_df.at[index, 'Opponent Ctrl'] = row['Ctrl']

            if row['Outcome'] == 'W':
                combined_df.at[index, 'Outcome'] = 'L'
            elif row['Outcome'] == 'L':
                combined_df.at[index, 'Outcome'] = 'W'

            combined_df.at[index, 'Sig. Landed'] = row['Opponent Sig. Landed']
            combined_df.at[index, 'Opponent Sig. Landed'] = row['Sig. Landed']     

            combined_df.at[index, 'Sig. Attempted'] = row['Opponent Sig. Attempted']
            combined_df.at[index, 'Opponent Sig. Attempted'] = row['Sig. Attempted']  

            combined_df.at[index, 'Total Landed'] = row['Opponent Total Landed']
            combined_df.at[index, 'Opponent Total Landed'] = row['Total Landed']   

            combined_df.at[index, 'Total Attempted'] = row['Opponent Total Attempted']
            combined_df.at[index, 'Opponent Total Attempted'] = row['Total Attempted']     

            combined_df.at[index, 'Td Landed'] = row['Opponent Td Landed']
            combined_df.at[index, 'Opponent Td Landed'] = row['Td Landed']

            combined_df.at[index, 'Td Attempted'] = row['Opponent Td Attempted']
            combined_df.at[index, 'Opponent Td Attempted'] = row['Td Attempted']           
    
    #ADD OHTER INTERESTING DATA TO BE USED IN ANALYSIS

    #Add current streak
    current_streak = []
    streak_type = []

    win_streak = 0
    loss_streak = 0

    for index, row in combined_df.iterrows():
        outcome = row['Outcome']

        if outcome == 'W':
            win_streak += 1
            loss_streak = 0
        elif outcome == 'L':
            loss_streak += 1
            win_streak = 0
        else:
            win_streak = 0
            loss_streak = 0

        if win_streak > 0:
            current_streak.append(win_streak)
            streak_type.append('Win')
        elif loss_streak > 0:
            current_streak.append(loss_streak)
            streak_type.append('Loss')
        else:
            current_streak.append(0)
            streak_type.append('None')


    combined_df['Current Streak'] = current_streak
    combined_df['Streak Type'] = streak_type

    # Add how long between each fight for analysis
    
    combined_df['Event Date'] = pd.to_datetime(combined_df['Event Date'])

    combined_df = combined_df.sort_values(by='Event Date', ascending=True)

    combined_df['Days since last fight'] = (combined_df['Event Date'] - combined_df['Event Date'].shift(1)).dt.days

    combined_df['Days since last fight'].fillna(0, inplace=True)


    return combined_df

#CREATE A FUNCTION TO CREATE A DF FROM FIGHTER_URL
def full_past_fights(fighter_url):
    past_fights_df = past_fights(fighter_url)  

    opponent_stats_df = get_opponent_stats(fighter_url)

    combined_df = pd.concat([past_fights_df, opponent_stats_df], axis=1)

    return combined_df

def convert_ctrl_to_seconds(ctrl_time):
    if pd.isna(ctrl_time):
        return np.nan
    minutes, seconds = map(int, ctrl_time.split(':'))
    return minutes * 60 + seconds

def preprocessing_data(fighter_url):
    fighter_df = full_past_fights(fighter_url)

    fighter_df.drop(['Fighter', 'Opponent', 'Event Name', 'Event Date', 'Time','Method of Victory', "Opponent STANCE:", "Streak Type"], axis=1, inplace=True)

    fighter_df['Ctrl'] = fighter_df['Ctrl'].replace('--', np.nan)
    fighter_df['Opponent Ctrl'] = fighter_df['Opponent Ctrl'].replace('--', np.nan)

    try:
        fighter_df['Ctrl (in seconds)'] = fighter_df['Ctrl'].apply(convert_ctrl_to_seconds)
        fighter_df['Opponent Ctrl (in seconds)'] = fighter_df['Opponent Ctrl'].apply(convert_ctrl_to_seconds)
        fighter_df.drop(['Ctrl', 'Opponent Ctrl'], axis=1, inplace=True)
    except ValueError:
        fighter_df['Ctrl (in seconds)'] = np.nan
        fighter_df['Opponent Ctrl (in seconds)'] = np.nan
    
    return fighter_df


In [ ]:
#TOP 10 FLYWEIGHTS URLS
flyweight_urls= [
'http://www.ufcstats.com/fighter-details/a0f0004aadf10b71',
'http://www.ufcstats.com/fighter-details/792be9a24df82ed6',
'http://www.ufcstats.com/fighter-details/6d35bf94f7d30241',
'http://www.ufcstats.com/fighter-details/853eb0dd5c0e2149',
'http://www.ufcstats.com/fighter-details/6e15f63b6c2e2c15',
'http://www.ufcstats.com/fighter-details/aa72b0f831d0bfe5',
'http://www.ufcstats.com/fighter-details/ab2b4ff41d6ebe0f',
'http://www.ufcstats.com/fighter-details/67c1d46f4ed16f9e',
'http://www.ufcstats.com/fighter-details/c96d9178c9ed9e62',
'http://www.ufcstats.com/fighter-details/5d1b7e3dd9e11074'   
]

#TOP 10 BANTAMWEIGHTS URLS
bantamweight_urls = [
'http://www.ufcstats.com/fighter-details/d661ce4da776fc20',
'http://www.ufcstats.com/fighter-details/b50a426a33da0012',
'http://www.ufcstats.com/fighter-details/cb696ebfb6598724',
'http://www.ufcstats.com/fighter-details/c03520b5c88ed6b4',
'http://www.ufcstats.com/fighter-details/056c493bbd76a918',
'http://www.ufcstats.com/fighter-details/65f09bacd3957381',
'http://www.ufcstats.com/fighter-details/7c7332319c14094c',
'http://www.ufcstats.com/fighter-details/efb96bf3e9ada36f',
'http://www.ufcstats.com/fighter-details/05339613bf8e9808',
'http://www.ufcstats.com/fighter-details/6bd02119599741a4']

#TOP 10 FEATHERWEIGHTS URLS
featherweight_urls= [
'http://www.ufcstats.com/fighter-details/e1248941344b3288',
'http://www.ufcstats.com/fighter-details/cbf5e6f231b55443',
'http://www.ufcstats.com/fighter-details/fba03cd6cc28dc41',
'http://www.ufcstats.com/fighter-details/54f64b5e283b0ce7',
'http://www.ufcstats.com/fighter-details/751de04455cfaac0',
'http://www.ufcstats.com/fighter-details/9560ff14eb3129f7',
'http://www.ufcstats.com/fighter-details/def8166ff24bd237',
'http://www.ufcstats.com/fighter-details/c451d67c09c55418',
'http://www.ufcstats.com/fighter-details/150ff4cc642270b9',
'http://www.ufcstats.com/fighter-details/76e2870ffafbe38f',
'http://www.ufcstats.com/fighter-details/040a74bb0a465c54']


#TOP 10 LIGHTWEIGHTS URLS
lightweight_urls= [
'http://www.ufcstats.com/fighter-details/275aca31f61ba28c',
'http://www.ufcstats.com/fighter-details/07225ba28ae309b6',
'http://www.ufcstats.com/fighter-details/9e8f6c728eb01124',
'http://www.ufcstats.com/fighter-details/029eaff01e6bb8f0',
'http://www.ufcstats.com/fighter-details/c814b4c899793af6',
'http://www.ufcstats.com/fighter-details/4b93a88f3b1de35b',
'http://www.ufcstats.com/fighter-details/08af939f41b5a57b',
'http://www.ufcstats.com/fighter-details/72db2a14ffa73ece',
'http://www.ufcstats.com/fighter-details/eae48ff31db420c2',
'http://www.ufcstats.com/fighter-details/99bd51917728c25d',
'http://www.ufcstats.com/fighter-details/193b9d1858bc4df3']

#TOP 10 WELTERWEIGHTS URLS
welterweight_urls= [
'http://www.ufcstats.com/fighter-details/f1fac969a1d70b08',
'http://www.ufcstats.com/fighter-details/f1b2aa7853d1ed6e',
'http://www.ufcstats.com/fighter-details/dc9572dd6ec74859',
'http://www.ufcstats.com/fighter-details/4a28cb716c19157a',
'http://www.ufcstats.com/fighter-details/b1b0729d27936f2f',
'http://www.ufcstats.com/fighter-details/01afe0916a40c7c5',
'http://www.ufcstats.com/fighter-details/23024fdfc966410a',
'http://www.ufcstats.com/fighter-details/b997be68943010fc',
'http://www.ufcstats.com/fighter-details/45f7cb591c3ab00b',
'http://www.ufcstats.com/fighter-details/6d4b63c767106d3a']


#TOP 10 MIDDLEWEIGHTS URLS
middleweight_urls= [
'http://www.ufcstats.com/fighter-details/1338e2c7480bdf9e',
'http://www.ufcstats.com/fighter-details/0d7b51c9d2649a6e',
'http://www.ufcstats.com/fighter-details/e1147d3d2dabe1ce',
'http://www.ufcstats.com/fighter-details/7acbb0972e75281a',
'http://www.ufcstats.com/fighter-details/2e5c2aa8e4ab9d82',
'http://www.ufcstats.com/fighter-details/b1a3e0aca758b322',
'http://www.ufcstats.com/fighter-details/327d5f279895110d',
'http://www.ufcstats.com/fighter-details/0a1942069c9ad6b6',
'http://www.ufcstats.com/fighter-details/2f181c0467965b98',
'http://www.ufcstats.com/fighter-details/13a0275fa13c4d26',
'http://www.ufcstats.com/fighter-details/0d8011111be000b2']

#TOP 10 LIGHT HEAVY WEIGHTS URLS
light_heavyweight_urls= [
'http://www.ufcstats.com/fighter-details/5444c5a201d3ee5a',
'http://www.ufcstats.com/fighter-details/e5549c82bfb5582d',
'http://www.ufcstats.com/fighter-details/c21f26bbde777573',
'http://www.ufcstats.com/fighter-details/99df7d0a2a08a8a8',
'http://www.ufcstats.com/fighter-details/333b9e5c723ac873',
'http://www.ufcstats.com/fighter-details/1091d4d957141094',
'http://www.ufcstats.com/fighter-details/d4c9dcd330403612',
'http://www.ufcstats.com/fighter-details/0845c81e37d3bcb3',
'http://www.ufcstats.com/fighter-details/a67f5afa8d6a1b80',
'http://www.ufcstats.com/fighter-details/009341ed974bad72',
'http://www.ufcstats.com/fighter-details/d802174b0c0c1f4e']

#TOP 10 HEAVYWEIGHTS URLS
heavyweight_urls = [
    'http://www.ufcstats.com/fighter-details/07f72a2a7591b409',
    'http://www.ufcstats.com/fighter-details/f14cf73e51b29254',
    'http://www.ufcstats.com/fighter-details/787bb1f087ccff8a',
    'http://www.ufcstats.com/fighter-details/d28dee5c705991df',
    'http://www.ufcstats.com/fighter-details/399afbabc02376b5',
    'http://www.ufcstats.com/fighter-details/fa6796c55d6c5440',
    'http://www.ufcstats.com/fighter-details/c62fbc117d57b943',
    'http://www.ufcstats.com/fighter-details/e2f6b2769aaedd6c',
    'http://www.ufcstats.com/fighter-details/279566840aa55bf2',
    'http://www.ufcstats.com/fighter-details/41e83a89929d1327',
    'http://www.ufcstats.com/fighter-details/d3df1add9d9a7efb'
]


In [ ]:
#Creating dataset for training model
def top_10_to_df(urls_list):
    weight_dfs = []

    for fighter_url in urls_list:
        try:
            weight_df = preprocessing_data(fighter_url)
            weight_dfs.append(weight_df)
        except Exception:
            pass

    combined_df = pd.concat(weight_dfs, ignore_index=True)

    return combined_df

flyweight_df  = top_10_to_df(flyweight_urls)
bantamweight_df = top_10_to_df(bantamweight_urls)
featherweight_df = top_10_to_df(featherweight_urls)
lightweight_df = top_10_to_df(lightweight_urls)
welterweight_df = top_10_to_df(welterweight_urls)
middleweight_df = top_10_to_df(middleweight_urls)
light_heavyweight_df = top_10_to_df(light_heavyweight_urls)
heavyweight_df = top_10_to_df(heavyweight_urls)

In [ ]:
#Training dataset
all_weight_df = pd.concat([flyweight_df, bantamweight_df, featherweight_df, lightweight_df, welterweight_df, middleweight_df, light_heavyweight_df, heavyweight_df], ignore_index=True)
all_weight_df.head()

In [ ]:
csv_file_path = 'all_weights.csv'

all_weight_df.to_csv(csv_file_path, index=False) 

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv("all_weights.csv")


In [4]:
#create model that can predict based on 'KD', 'Opponent KD', 'Sig. str. %', 'Opponent Sig. str. %', 'Td %', 'Opponent Td %', 'Sub. att', 'Opponent Sub. att', 'Rev.', 'Opponent Rev.]]  # Include all relevant columns
df['Outcome'] = df['Outcome'].apply(lambda x: 1 if x == 'W' else (0 if x == 'L' else np.nan))
df.dropna(subset=['Outcome'], inplace=True)

df.fillna(df.mean(), inplace=True)



In [6]:
#Training Model
X = df[['KD', 'Opponent KD', 'Sig. str. %', 'Opponent Sig. str. %', 'Td %', 'Opponent Td %', 'Sub. att', 'Opponent Sub. att', 'Rev.', 'Opponent Rev.']]  
y = df['Outcome']
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
from sklearn.metrics import accuracy_score, classification_report


y_pred = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)



Accuracy: 0.845771144278607
Classification Report:
               precision    recall  f1-score   support

         0.0       0.75      0.68      0.71        56
         1.0       0.88      0.91      0.89       145

    accuracy                           0.85       201
   macro avg       0.81      0.79      0.80       201
weighted avg       0.84      0.85      0.84       201



In [7]:
#Saving model
import joblib

model_filename = 'fighter_prediction_model.pkl'
joblib.dump(rf_model, model_filename)


['fighter_prediction_model.pkl']

In [8]:
#CREATING Flask app to receive user's input and make prediction

from flask import Flask, render_template, request
import pandas as pd  

model_filename = 'fighter_prediction_model.pkl'
loaded_model = joblib.load(model_filename)


app = Flask(__name__)

@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        # Get user input from the form
        user_input_kd = float(request.form["kd"])
        user_input_opponent_kd = float(request.form["opponent_kd"])
        user_input_sig_str = float(request.form["user_input_sig_str"])
        user_input_opponent_sig_str = float(request.form["user_input_opponent_sig_str"])
        user_input_td = float(request.form["user_input_td"])
        user_input_opponent_td = float(request.form["user_input_opponent_td"])
        user_input_sub_att = float(request.form["user_input_sub_att"])
        user_input_opponent_sub_att = float(request.form["user_input_opponent_sub_att"])
        user_input_rev = float(request.form["user_input_rev"])
        user_input_opponent_rev = float(request.form["user_input_opponent_rev"])

        # Create a user data DataFrame
        user_data = pd.DataFrame({
            'KD': [user_input_kd],
            'Opponent KD': [user_input_opponent_kd],
            'Sig. str. %': [user_input_sig_str],
            'Opponent Sig. str. %': [user_input_opponent_sig_str],
            'Td %': [user_input_td],
            'Opponent Td %': [user_input_opponent_td],
            'Sub. att': [user_input_sub_att],
            'Opponent Sub. att': [user_input_opponent_sub_att],
            'Rev.': [user_input_rev],
            'Opponent Rev.': [user_input_opponent_rev],
        })

        prediction = rf_model.predict(user_data)
        result = "Win" if prediction[0] == 1 else "Loss"
        return render_template("index.html", result=result)

    return render_template("index.html", result=None)

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

C:\Users\Bootcamp\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3450: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
